## Part 1 - Data Analysis and Bayes Nets

### 1) Data Visualization and Exploration

Importing the required libraries and checking for the python version.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3,5)

In [ ]:
# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

In [ ]:
# Common imports
import numpy as np
import pandas as pd
import os
import cv2  # OpenCV Python library for computer vision

# to make this notebook's output stable across runs
np.random.seed(42)

In [ ]:
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
import random

Loading only the training set for Part 1 of the coursework

In [ ]:
CW_DATASET_PATH = "CW_dataset"

def load_train_data(dataset_path=CW_DATASET_PATH):
    x_train_all_path = os.path.join(dataset_path, "x_train_all.csv")
    y_train_all_path = os.path.join(dataset_path, "y_train_all.csv")

    x_train_all = pd.read_csv(x_train_all_path)
    y_train_all = pd.read_csv(y_train_all_path)

    return x_train_all, y_train_all

In [ ]:
x_train, y_train = load_train_data()

In [ ]:
x_train.shape

Analysing the dataset

In [ ]:
#Checking the shape of the data
x_train.shape

In [ ]:
y_train.shape

In [ ]:
#Displaying the first 5 rows of the dataset
x_train.head()

In [ ]:
y_train.head()

In [ ]:
#Checking for missing values
x_train.isnull().sum

In [ ]:
y_train.isnull().sum

In [ ]:
#To check if there are any missing values in the data frame
x_train.isna().any().any()

In [ ]:
y_train.isna().any().any()

In [ ]:
# Get a summary of the dataset using describe
x_train.describe()

In [ ]:
y_train.describe()

In [ ]:
label_counts = y_train['0'].value_counts().sort_index()
label_counts

Visualising the dataset using graphs

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='0', data=y_train)
plt.title("Distribution of Class Labels")
plt.xlabel("Class Labels")
plt.ylabel("Count")
plt.show()

In [ ]:
# To display one image for each label

# Initialize a dictionary to store one image for each label
label_images = {}

# Iterate through the rows of the DataFrames and find one image for each label
for index, row in x_train.iterrows():
    label = y_train.iloc[index, 0]  
    
    if label not in label_images:
        # Store the first image for each unique label
        label_images[label] = row.values.reshape(48, 48)  
    
    # Break the loop if we have found one image for each unique label
    if len(label_images) == 10:
        break

# Display the images
fig, axs = plt.subplots(2, 5, figsize=(12, 6))
for i, (label, image) in enumerate(label_images.items()):
    r, c = divmod(i, 5)
    axs[r, c].imshow(image)
    axs[r, c].set_title(f'Label: {label}')
    axs[r, c].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

unique_class_labels = y_train['0'].unique()

fig, axes = plt.subplots(1, len(unique_class_labels), figsize=(20, 5))

for i, class_label in enumerate(unique_class_labels):
    # Select an image for each class
    class_images = x_train[y_train['0'] == class_label]
    
    if not class_images.empty:
        img = class_images.iloc[0, :-1].values
        
        # Plot the histogram for the image of each class
        axes[i].hist(img, bins=100)
        axes[i].set_title(f'Class {class_label}')
        axes[i].set_xlabel("Pixel Value")
        axes[i].set_ylabel("Frequency")

plt.show()


In [ ]:
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)

# store image data for each label
label_images = {}

# just to check if all images are being plotted
num_images_per_label = {}

for index, row in x_train.iterrows():
    label = y_train.iloc[index, 0]  
    image_data = row.values.reshape(-1, 48, 48) 

    if label not in label_images:
        label_images[label] = []
    
    # add image data to associated labels
    label_images[label].append(image_data)

    # just to check if all images are being plotted
    num_images_per_label[label] = len(label_images[label])

# get numpy array from dictionary values (image data)
box_data = np.array(list(label_images.values()), dtype=object)

# labels from keys
labels = list(label_images.keys())

# box plots for each label 
plt.figure(figsize=(12, 6))
plt.boxplot(box_data, labels=labels, vert=False)
plt.title("Box Plots for each label")
plt.xlabel("Pixel Values")
plt.ylabel("Label")
plt.show()


# just to check if all images are being plotted
for label, num_images in num_images_per_label.items():
    print(f"Label {label}: {num_images} images")

In [ ]:
onevrall_x_train = pd.DataFrame(x_train) 
no_aug_y_train = pd.DataFrame(y_train)
print(x_train.shape, y_train.shape)

### 2) Preprocessing the Train Dataset

#### 2.1) Categorical Naïve Bayes Classifier

In [ ]:
# Created a list to store the labels that have less number of pictures
less_class_labels = [0, 5, 6, 7, 8, 9]

data_list = []
for index, row in x_train.iterrows():
    label = y_train.iloc[index, 0]
    image_data = row.values.reshape(48, 48)

    if label in less_class_labels:
        data_list.append((image_data, label))

subset_dataset = np.array(data_list, dtype=object)

print(subset_dataset.shape)

num_images_per_label = {label: np.sum(subset_dataset[:, 1] == label) for label in less_class_labels}

for label, num_images in num_images_per_label.items():
    print(f"Label {label}: {num_images} images")

In [ ]:
def add_noise(feature_vector, noise_level=0.01):
    noise = np.random.normal(0, noise_level, feature_vector.shape)
    noisy_vector = feature_vector + noise
    return noisy_vector

In [ ]:
def introduce_perturbations(feature_vector, perturbation_level=0.02):
    min_value = np.min(feature_vector)
    max_value = np.max(feature_vector)
    perturbations = np.random.uniform(-perturbation_level, perturbation_level, feature_vector.shape)
    perturbed_vector = feature_vector + perturbations * (max_value - min_value)
    return perturbed_vector

In [ ]:
print(x_train.shape)
print(y_train.shape)
print('------------------------')

augmented_data = []
new_augmented_data = []
labels_for_train = []
for data, label in subset_dataset:
    noisy_vector = add_noise(data, noise_level=0.01)  
    perturbed_vector = introduce_perturbations(data, perturbation_level=0.02)  
    
        # Append the original and augmented data with their respective labels
    augmented_data.append((data, label))
    augmented_data.append((noisy_vector, label))
    augmented_data.append((perturbed_vector, label))

# Convert the appended data to a NumPy array
for image_vectors, label_col in augmented_data:
    new_augmented_data.append(image_vectors)
    labels_for_train.append(label_col)

new_augmented_data = np.array(new_augmented_data)
labels_for_train = np.array(labels_for_train)

new_labels = labels_for_train.reshape(-1, 1)

new_augmented_data = new_augmented_data.reshape(new_augmented_data.shape[0], -1)

print(new_augmented_data.shape)
print(new_labels.shape)
print('------------------------')
if new_augmented_data.shape[1] == x_train.shape[1]:
    cnb_x_train = np.vstack((x_train, new_augmented_data))
    cnb_y_train = np.concatenate((y_train, new_labels))
else:
    print("Number of columns in appended_data doesn't match x_train.")

print(cnb_x_train.shape)
print(cnb_y_train.shape)

#### 2.1) Gaussian Naïve Bayes Classifier

In [ ]:
x_train = np.array(x_train)

def normalise(images):
    preprocessed_images = []
    for image in images: 
        if(len(image.shape) == 3):
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        if image.dtype != np.uint8:
            image = image.astype(np.uint8)

        adjusted = cv2.convertScaleAbs(image, alpha=1.7, beta = 90)
        enhanced_image = cv2.equalizeHist(adjusted)
        image = enhanced_image/255.0
        resize = cv2.resize(image, (48,48))

        preprocessed_images.append(resize)
    return  np.array(preprocessed_images)


x_train = normalise(x_train)

x_train = x_train.reshape(x_train.shape[0], -1)
x_train.shape

In [ ]:
print(x_train.shape)
print(y_train.shape)

augmented_data = []
new_augmented_data = []
labels_for_train = []
for data, label in subset_dataset:
    noisy_vector = add_noise(data, noise_level=0.01)  
    perturbed_vector = introduce_perturbations(data, perturbation_level=0.02)  
    
        # Append the original and augmented data with their respective labels
    augmented_data.append((data, label))
    augmented_data.append((noisy_vector, label))
    augmented_data.append((perturbed_vector, label))

# Convert the appended data to a NumPy array
for image_vectors, label_col in augmented_data:
    new_augmented_data.append(image_vectors)
    labels_for_train.append(label_col)

new_augmented_data = np.array(new_augmented_data)
labels_for_train = np.array(labels_for_train)

new_labels = labels_for_train.reshape(-1, 1)

new_augmented_data = new_augmented_data.reshape(new_augmented_data.shape[0], -1)

print(new_augmented_data.shape)
print(new_labels.shape)
if new_augmented_data.shape[1] == x_train.shape[1]:
    x_train = np.vstack((x_train, new_augmented_data))
    y_train = np.concatenate((y_train, new_labels))
else:
    print("Number of columns in appended_data doesn't match x_train.")

print(x_train.shape)
print(y_train.shape)

In [ ]:
# import numpy as np
# from skimage import io, transform, img_as_ubyte
# import matplotlib.pyplot as plt
# import pandas as pd

# x_train = pd.DataFrame(x_train)
# y_train = pd.DataFrame(y_train)

# preprocessed_images = []

# for index, row in x_train.iterrows():
#     label = y_train.iloc[index, 0]

    
#     # Reshape the image data to its original shape (assuming it's a 48x48 image)
#     image = row.values.reshape(48, 48)

#     # Define the range of rotation angles (e.g., -10 to 10 degrees)
#     min_angle = -10
#     max_angle = 10
#     rotation_angle = np.random.uniform(min_angle, max_angle)

#     # Define a random affine transformation matrix with rotation
#     tform = transform.AffineTransform(rotation=np.deg2rad(rotation_angle))

#     # Apply the affine transformation to the image
#     J2 = transform.warp(image, tform)

#     # Convert the transformed image to 8-bit format (if necessary)
#     # J2 = img_as_ubyte(J2)

#     # Display the original and transformed images
#     plt.figure(figsize=(8, 4))
#     plt.subplot(1, 2, 1)
#     plt.title('Original Image')
#     plt.imshow(image)
#     plt.axis('off')

#     plt.subplot(1, 2, 2)
#     plt.title('Transformed Image')
#     plt.imshow(J2)
#     plt.axis('off')

#     plt.show()

#     fig, axs = plt.subplots(2, 5, figsize=(12, 6))
#     for i, (label, image) in enumerate(label_images.items()):
#     r, c = divmod(i, 5)
#     axs[r, c].imshow(image)
#     axs[r, c].set_title(f'Label: {label}')
#     axs[r, c].axis('off')

#     plt.tight_layout()
#     plt.show()

# x_train_rotated = np.array(preprocessed_images)

# # # Reshape the equalized images if necessary
# x_train_rotated = x_train_rotated.reshape(x_train_rotated.shape[0], -1)

# print(x_train_rotated.shape)

In [ ]:
# x_train = x_train_rotated
# x_train

### 3) Running Naïve Bayes Classifier on Train Dataset

#### 3.1) Categorical Naïve Bayes Classifier

In [ ]:
from sklearn.naive_bayes import CategoricalNB

cnb_x_train = np.asarray(cnb_x_train)
xtrain_images = cnb_x_train.reshape(cnb_x_train.shape[0], -1)
cnb_y_train = np.asarray(cnb_y_train).ravel()
clf = CategoricalNB()
clf.fit(xtrain_images,cnb_y_train)

predictions = clf.predict(cnb_x_train)

#### 3.2) Gaussian Naïve Bayes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB

x_train = np.asarray(x_train)
xtrain_images = x_train.reshape(x_train.shape[0], -1)
y_train = np.asarray(y_train).ravel()
clf2 = GaussianNB()
clf2.fit(xtrain_images, y_train)

predictions_gb = clf2.predict(x_train)

### 4) Evaluation Metrics for the Naïve Bayes Classifier on Dataset

#### 4.1) Categorical Naïve Bayes Classifier

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(cnb_y_train, predictions)

print(report)

In [ ]:
from sklearn.metrics import accuracy_score

# accuracy
accuracy = accuracy_score(cnb_y_train, predictions)
print('Accuracy = ', accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(cnb_y_train, predictions)
confusion_matrix

In [ ]:
import seaborn as sns
plt.figure(figsize=(10, 8)) 
sns.heatmap(confusion_matrix/np.sum(confusion_matrix), annot=True,  fmt='.2%', cmap='Blues')

In [ ]:
tp = confusion_matrix.diagonal()
fn = np.sum(confusion_matrix, axis=1) - tp
fp = np.sum(confusion_matrix, axis=0) - tp
tn = np.sum(confusion_matrix) - (tp + fn + fp)

# TP Rate (Sensitivity or Recall)
tp_rate = tp / (tp + fn)

# FP Rate
fp_rate = fp / (fp + tn)

for class_label, tp, fp in zip(range(len(tp_rate)), tp_rate, fp_rate):
    print(f"Class {class_label}: \nTP Rate = {tp}, \nFP Rate = {fp}")

In [ ]:
specificity = []
unique_labels = np.unique(cnb_y_train)

for i in range(len(unique_labels)):
    true_negative = np.sum(confusion_matrix) - np.sum(confusion_matrix[i, :]) - np.sum(confusion_matrix[:, i]) + confusion_matrix[i, i]
    total_negative = np.sum(confusion_matrix) - np.sum(confusion_matrix[i, :])
    print(f'Label {unique_labels[i]} specificity: {true_negative / total_negative}')

In [ ]:
from sklearn.metrics import recall_score

sensitivity = recall_score(cnb_y_train, predictions, average=None)
for i in range(len(unique_labels)):
    print(f'Label {unique_labels[i]} sensitivity: {sensitivity[i]}')

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

probabilities = clf.predict_proba(cnb_x_train)
roc_auc_scores = []
plt.figure(figsize=(8, 6))

for i in range(len(unique_labels)):
    fpr, tpr, _ = roc_curve((cnb_y_train == unique_labels[i]).astype(int), probabilities[:, i])
    roc_auc = auc(fpr, tpr)
    roc_auc_scores.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
    
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

##### Evaluation Using Cross-Validation

We had manually created our own cross-validation which divides our dataset into folds and iteratively trains and evaluates our Categorical NB classifier on each fold. We also tried the cross-validation technique of sklearn module.

But we found that we kept having an error with the cross-validation even though we achieved high accuracy. There were 2 reasons:

1) Our dataset only had Numeric/Continuous data and had no categorical data. We later discovered the actual purpose of a Categorical NB Classifier through this evaluation, that is Categorical NB can only be used categorical data like gender, color, etc.

In [ ]:
data_types = pd.DataFrame(cnb_x_train).dtypes
numeric_columns = data_types[data_types.apply(lambda x: pd.api.types.is_numeric_dtype(x))].index
categorical_columns = data_types[data_types.apply(lambda x: not pd.api.types.is_numeric_dtype(x))].index

print("Numeric (Continuous) Columns:")
print(numeric_columns)

print("\nCategorical Columns:")
print(categorical_columns)

2. The 2nd reason is listed below:

In [ ]:
x_num_features = cnb_x_train.shape[1]
num_features_expected = len(clf.n_categories_)
list_features = clf.n_categories_
print(f"The CategoricalNB model takes {x_num_features} as input and as shown expects {num_features_expected} features.")
print(f"But if you look at the actual categories,", list_features)
print(f"It is converting each of our images into these categories: ", list(set(list_features)))
print("Due to this, we had an error for cross-validation. And due to this we discovered how categorical NB")
print("actually works and why we were having a very good accuracy.")

#### 4.2) Gaussian Naïve Bayes Classifier

In [ ]:
from sklearn.metrics import classification_report

report_gb = classification_report(y_train, predictions_gb)

print(report_gb)

In [ ]:
from sklearn.metrics import accuracy_score

# accuracy
accuracy_gb = accuracy_score(y_train, predictions_gb)
print('Accuracy = ', accuracy_gb)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix_gb = confusion_matrix(y_train, predictions_gb)
confusion_matrix_gb

In [ ]:
import seaborn as sns
plt.figure(figsize=(10, 8)) 
sns.heatmap(confusion_matrix_gb/np.sum(confusion_matrix_gb), annot=True,  fmt='.2%', cmap='Blues')

In [ ]:
tp_gb = confusion_matrix_gb.diagonal()
fn_gb = np.sum(confusion_matrix_gb, axis=1) - tp
fp_gb = np.sum(confusion_matrix_gb, axis=0) - tp
tn_gb = np.sum(confusion_matrix_gb) - (tp + fn + fp)

# TP Rate (Sensitivity or Recall)
tp_rate_gb = tp_gb / (tp_gb + fn_gb)

# FP Rate
fp_rate_gb = fp_gb / (fp_gb + tn_gb)

for class_label, tp, fp in zip(range(len(tp_rate_gb)), tp_rate_gb, fp_rate_gb):
    print(f"Class {class_label}: \nTP Rate = {tp}, \nFP Rate = {fp}")

In [ ]:
specificity_gb = []
unique_labels_gb = np.unique(y_train)

for i in range(len(unique_labels_gb)):
    true_negative_gb = np.sum(confusion_matrix_gb) - np.sum(confusion_matrix_gb[i, :]) - np.sum(confusion_matrix_gb[:, i]) + confusion_matrix_gb[i, i]
    total_negative_gb = np.sum(confusion_matrix_gb) - np.sum(confusion_matrix_gb[i, :])
    print(f'Label {unique_labels_gb[i]} specificity: {true_negative_gb / total_negative_gb}')

In [ ]:
from sklearn.metrics import recall_score

sensitivity_gb = recall_score(y_train, predictions_gb, average=None)
for i in range(len(unique_labels_gb)):
    print(f'Label {unique_labels_gb[i]} sensitivity: {sensitivity_gb[i]}')

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

probabilities_gb = clf2.predict_proba(x_train)
roc_auc_scores_gb = []
plt.figure(figsize=(8, 6))

for i in range(len(unique_labels_gb)):
    fpr_gb, tpr_gb, _ = roc_curve((y_train == unique_labels_gb[i]).astype(int), probabilities_gb[:, i])
    roc_auc_gb = auc(fpr_gb, tpr_gb)
    roc_auc_scores_gb.append(roc_auc_gb)
    plt.plot(fpr_gb, tpr_gb, lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc_gb))
    
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

##### Evaluation Using Cross-Validation

In [ ]:
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [ ]:
scores = cross_val_score(clf2, x_train, y_train, scoring="accuracy", cv=10)
display_scores(scores)

In [ ]:
scores = cross_val_score(clf2, x_train, y_train, scoring="neg_mean_squared_error", cv=10)
clf2_rmse_scores = np.sqrt(-scores)
display_scores(clf2_rmse_scores)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

seed_value = 24

x_train_split, x_test_split, y_train_split, y_test_split = train_test_split(x_train, y_train, test_size=0.3, random_state=seed_value)

gnb = GaussianNB()
gnb.fit(x_train_split, y_train_split)

predicted = gnb.predict(x_test_split)

In [ ]:
accuracy = accuracy_score(y_test_split, predicted)
accuracy

In [ ]:
f1 = f1_score(y_test_split, predicted, average='macro')
f1

In [ ]:
conf_matrix = confusion_matrix(y_test_split, predicted)
conf_matrix

### 5) Top Correlating Features - Feature Selection

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

# Defined the number of features to select for each class
no_of_features_per_class = [5, 10, 20]

# Dictionary stores the top features for each class and each dataset
top_features_per_cd = {}

x_train_class_vs_rest = onevrall_x_train

for class_label in range(10):
    onevrsall_path = os.path.join("OnevrsAll", f"{class_label}_vrs_all")
    y_train_path = os.path.join(onevrsall_path, f"y_train_{class_label}.csv")
    y_train_class_vs_rest = pd.read_csv(y_train_path)
    
    # Dictionary to store the top features for the current class
    top_features_per_class = {}
    
    # Trained a SVC for the one-vs-rest classifier
    classifier = OneVsRestClassifier(SVC(kernel='linear'))
    classifier.fit(x_train_class_vs_rest, y_train_class_vs_rest)
    
    # Get the coefficients (weights) for the features
    feature_weights = classifier.estimators_[0].coef_[0]
    # Sort the features by their absolute weights and select the top features
    for no_f in no_of_features_per_class:
        top_feature_indices = np.argsort(np.abs(feature_weights))[::-1][:no_f]
        top_features = x_train_class_vs_rest.columns[top_feature_indices]
        top_features_per_class[no_f] = top_features.tolist()

    top_features_per_cd[class_label] = top_features_per_class

final_datasets = {}
for i in range(10):
    for no_f in no_of_features_per_class:
        dataset_name = f"Data set {no_f}({i})"
        
        final_datasets[dataset_name] = pd.DataFrame(x_train_class_vs_rest[top_features_per_cd[i][no_f]])
print("30 Datasets for each class and for each top features [5,10,20]: \n", final_datasets)

final_3_datasets = {}
for no_f in no_of_features_per_class:
    dataset_name = f"Data set {no_f}"
    
    selected_features = []
    for class_label, top_features in top_features_per_cd.items():
        selected_features.extend(top_features[no_f])
    
    final_3_datasets[dataset_name] = x_train_class_vs_rest[selected_features]
print("3 Datasets Combined: \n", final_3_datasets)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

seed_value = 24

accuracies = []

for i in final_datasets:
    no_aug_y_train = np.asarray(no_aug_y_train).ravel()
    x_train_split, x_test_split, y_train_split, y_test_split = train_test_split(final_datasets[i], no_aug_y_train, test_size=0.3, random_state=seed_value)

    gnb = GaussianNB()
    gnb.fit(x_train_split, y_train_split)

    predictions_gb = gnb.predict(x_test_split)
    
    accuracy_gb = accuracy_score(y_test_split, predictions_gb)
    accuracies.append(accuracy_gb)
    print(f'Accuracy for {i} = ', accuracy_gb)
accuracies = np.array(accuracies)
print("Mean Accuracy = ", accuracies.mean())

In [ ]:
dataset1 = final_3_datasets['Data set 5']
dataset2 = final_3_datasets['Data set 10']
dataset3 = final_3_datasets['Data set 20']

In [ ]:
dataset1

### 6) Run Naïve Bayes Classifier on data sets created & Evaluation Metrics

Run the Gaussian Naïve Bayes Classifier on the 30 datasets created for each class and for each top correlating features 
[5,10,20].

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_score
import numpy as np

class_labels = list(range(10))

def evaluate_guassian_nb(X_train, y_train):
    nb_model = GaussianNB()
    nb_model.fit(X_train, y_train)
    y_pred = nb_model.predict(X_train)
    accuracy = accuracy_score(y_train, y_pred)
    precision = precision_score(y_train, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_train, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_train, y_pred, average='macro', zero_division=1)
    roc_auc = roc_auc_score(y_train, nb_model.predict_proba(X_train), multi_class='ovr')
    cross_val_scores = cross_val_score(nb_model, X_train, y_train, scoring="accuracy", cv=10)
    return accuracy, precision, recall, f1, roc_auc, cross_val_scores

y_train_dict = {}
evaluation_metrics = []
dataset_number_features = [5, 10, 20]

for i in final_datasets:
    X_train_fr = final_datasets[i]
    
    accuracy, precision, recall, f1, roc_auc, cross_val_scores = evaluate_guassian_nb(X_train_fr, no_aug_y_train)

    evaluation_metrics.append({
        "Dataset": i,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "ROC AUC": roc_auc,
        "Cross Val Scores": np.mean(cross_val_scores),
        "TP rate": recall, 
        "FP rate": 1 - recall
    })

df_evaluation_metrics = pd.DataFrame(evaluation_metrics)

In [ ]:
df_evaluation_metrics

Run the Gaussian Naïve Bayes Classifier on the final 3 combined datasets created of each class [5,10,20].

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_score
import numpy as np

class_labels = list(range(10))

def evaluate_gaussian_nb(X_train, y_train):
    nb_model = GaussianNB()
    nb_model.fit(X_train, y_train)
    y_pred = nb_model.predict(X_train)
    accuracy = accuracy_score(y_train, y_pred)
    precision = precision_score(y_train, y_pred)
    recall = recall_score(y_train, y_pred)
    f1 = f1_score(y_train, y_pred)
    roc_auc = roc_auc_score(y_train, y_pred)
    cross_val_scores = cross_val_score(nb_model, X_train, y_train, scoring="accuracy", cv=10)
    return accuracy, precision, recall, f1, roc_auc, cross_val_scores

datasets = [dataset1, dataset2, dataset3]  
y_train_dict = {}
evaluation_metrics = []
dataset_number_features = [5, 10, 20]

for dataset_number, dataset in enumerate(datasets):
    X_train_fr = dataset
    
    for class_label in class_labels:
        onevrsall_path = os.path.join("OnevrsAll", f"{class_label}_vrs_all")
        y_train_file = os.path.join(onevrsall_path, f"y_train_{class_label}.csv")
        y_train_df = pd.read_csv(y_train_file).values.ravel()
    
        accuracy, precision, recall, f1, roc_auc, cross_val_scores = evaluate_gaussian_nb(X_train_fr, y_train_df)

        evaluation_metrics.append({
            "Dataset": str(dataset_number_features[dataset_number]),
            "Class Label": class_label,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1,
            "ROC AUC": roc_auc,
            "Cross Val Scores": np.mean(cross_val_scores),
            "TP rate": recall, 
            "FP rate": 1 - recall
        })

df3_evaluation_metrics = pd.DataFrame(evaluation_metrics)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC']

for metric in metrics:
    plt.figure(figsize=(4, 4)) 
    for dataset_number in [5, 10, 20]:
        dataset_metrics = df3_evaluation_metrics[df3_evaluation_metrics['Dataset'] == str(dataset_number)]
        x_values = dataset_metrics['Class Label']
        y_values = dataset_metrics[metric]
        plt.plot(x_values, y_values, marker='o', label=f'Dataset {dataset_number}')

    plt.title(f'{metric} by Class Label for Different Datasets')
    plt.xlabel('Class Label')
    plt.ylabel(metric)
    plt.legend()
    plt.xticks(x_values)  
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
df3_evaluation_metrics

### Explored the Coffee Leaf dataset that we used in Lab 4 and Lab 5

#### JMuBEN dataset

### 1) Loading the dataset

In [ ]:
import os
import requests
import zipfile
import numpy as np

dataset_url = 'https://github.com/ishaqmarashy/DATALFS/raw/main/JMuBEN.zip'
dataset_dir = './JMuBEN'

# create directory for dataset if it does not exist
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
    
# append JMuBEN.zip to the end of the path (this is where we download the file to)
zip_file_path = os.path.join(dataset_dir, 'JMuBEN.zip')


# check if file is downloaded already
if not os.path.exists(zip_file_path):
    
    # file is not downloaded so fetch the file
    response = requests.get(dataset_url)
    
    # write file to storage which is recieved from the response
    with open(zip_file_path, 'wb') as zip_file:
        zip_file.write(response.content)
        
    # unzip to zip file path
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(dataset_dir)

# within the concat train and test to become ./JMuBEN/train and JMuBEN ./JMuBEN/test
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')

In [ ]:
def load_images_and_labels(directory):
    images = []
    labels = []
    
    # get subdirectories Healthy and Miner
    
    for class_name in os.listdir(directory):
        
        # concat subdirectory to get full path
        class_dir = os.path.join(directory, class_name)
        # assign labels using class subdirectory
        # label is determined by filepath
        label = 0 if class_name == 'Miner' else 1
        
        # append labels and image paths to labels and images respectively
        for filename in os.listdir(class_dir):
            images.append(os.path.join(class_dir, filename))
            labels.append(label)
            
    return images, labels

# load file directories and their labels
train_images_dir, train_labels = load_images_and_labels(train_dir)
test_images_dir, test_labels = load_images_and_labels(test_dir)

In [ ]:
# print the number of images and labels

print(f"Train images:{len(train_images_dir)}  Labels:{len(train_labels)}")
print(f"Test images:{len(test_images_dir)}  Labels:{len(test_labels)}")

#### 2) Pre-processing

In [ ]:
import cv2
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

def load_grayscale_images(image_paths):
    loaded_images = []
    for image_path in image_paths:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  #  grayscale
        loaded_images.append(img)
    return loaded_images

def resize_images(images_to_resize):
    resized_images = []
    for img in images_to_resize:
        resized_img = cv2.resize(img, (128, 128))  # resize
        resized_images.append(resized_img)
    return resized_images

def normalize_image(images_to_normalize):
    normalized_images = []
    for img in images_to_normalize:
        normalized_img = img / 255.0  # normalize
        normalized_images.append(normalized_img)
    return normalized_images

image_pipeline = Pipeline(steps=[
    ('load_grayscale_images', FunctionTransformer(load_grayscale_images)),
    ('resize_images', FunctionTransformer(resize_images)),
    ('normalize_image', FunctionTransformer(normalize_image))
])


train_images=image_pipeline.transform(train_images_dir)
test_images=image_pipeline.transform(test_images_dir)

In [ ]:
print(len(train_images))
print(len(test_images))

#### 3) Data Visualisation

In [ ]:
import matplotlib.pyplot as plt

sample_image = train_images[3]

plt.figure(figsize=(8, 6))
plt.imshow(sample_image, cmap='viridis')  
plt.colorbar()
plt.title('Heatmap of Pixel Values')
plt.show()

In [ ]:
import cv2
import numpy as np

image = train_images[3]
pixel_values = image.reshape(-1)  # Flatten the image into a 1D array
mean_pixel_value = np.mean(pixel_values)
std_pixel_value = np.std(pixel_values)
min_pixel_value = np.min(pixel_values)
max_pixel_value = np.max(pixel_values)

 
print(f"Mean Pixel Value: {mean_pixel_value}")
print(f"Standard Deviation of Pixel Values: {std_pixel_value}")
print(f"Minimum Pixel Value: {min_pixel_value}")
print(f"Maximum Pixel Value: {max_pixel_value}")

In [ ]:
from collections import Counter

# Calculate class counts
class_counts_train= Counter(train_labels)
print(class_counts_train)

class_counts_test = Counter(test_labels)
print(class_counts_test)

In [ ]:
import matplotlib.pyplot as plt

# Example: Histogram of class distribution
plt.hist(train_labels, bins=[0, 1,2], align='left', rwidth=0.6)
plt.xticks([0, 1], ['0', '1'])
plt.xlabel('Class Label')
plt.ylabel('Count')
plt.title('Class Distribution for train dataset')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Example: Histogram of class distribution
plt.hist(test_labels, bins=[0, 1,2], align='left', rwidth=0.6)
plt.xticks([0, 1], ['0', '1'])
plt.xlabel('Class Label')
plt.ylabel('Count')
plt.title('Class Distribution for test dataset')
plt.show()

In [ ]:
import os
import cv2
import numpy as np

# Get the current working directory as the root path
root_path = os.path.join(os.getcwd(), "JMuBEN")

# Initialize empty lists to store the information
sizes = []
resolutions = []
color_distributions = []

# Iterate over each image file in each subdirectory
for dirpath, dirnames, filenames in os.walk(root_path):
    for filename in filenames:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Load the image file using OpenCV
            img_path = os.path.join(dirpath, filename)
            img = cv2.imread(img_path)

            # Extract the size of the image
            size = os.path.getsize(img_path)
            sizes.append(size)

            # Extract the resolution of the image
            resolution = img.shape[:2]
            resolutions.append(resolution)

            # Extract the color distribution of the image
            color_distribution = np.bincount(img.flatten(), minlength=256)
            color_distributions.append(color_distribution)

# Convert the lists to numpy arrays for easier manipulation
sizes = np.array(sizes)
resolutions = np.array(resolutions)
color_distributions = np.array(color_distributions)

In [ ]:
import matplotlib.pyplot as plt

# Calculate the mean color distribution across all images
mean_color_distribution = np.mean(color_distributions, axis=0)

# Plot a bar chart of the mean color distribution
plt.bar(np.arange(256), mean_color_distribution)
plt.title("Mean Color Distribution of the dataset")
plt.xlabel("Colour Value (Combined RGB)")
plt.ylabel("Number of Pixels")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot a histogram of the image sizes
plt.hist(sizes)
plt.title("Distribution of Image Sizes ")
plt.xlabel("File Size (bytes)")
plt.ylabel("Number of Images")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot a scatter plot of the image resolutions
plt.scatter(resolutions[:, 0], resolutions[:, 1])
plt.title("Distribution of Image Resolutions")
plt.xlabel("Width (pixels)")
plt.ylabel("Height (pixels)")
plt.show()

#### 4) Exploring Image Processing Techniques

##### · Canny Edge Detection

In [ ]:
#Canny Edge Detection

import cv2 as cv
from matplotlib import pyplot as plt


#replace image path as needed 
img_path  = os.path.join(os.getcwd(), "JMuBEN/train/Miner/1 (4471).jpg")
img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
assert img is not None, "file could not be read, check with os.path.exists()"
edges = cv.Canny(img,100,200)
plt.subplot(121),plt.imshow(img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
plt.show()

##### · Image Segmentation

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


image_grayscale = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
image_normal = cv2.imread(img_path)

#threshold to segment the image
threshold_value = 130  
ret, thresholded_image = cv2.threshold(image_grayscale, threshold_value, 255, cv2.THRESH_BINARY)
ret, thresholded_image_coloured = cv2.threshold(image_normal, threshold_value, 255, cv2.THRESH_BINARY)

# Display 
plt.figure(figsize=(10, 5))
plt.subplot(131), plt.imshow(image, cmap='gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(132), plt.imshow(thresholded_image, cmap='gray')
plt.title('Thresholded Image'), plt.xticks([]), plt.yticks([])
plt.subplot(133), plt.imshow(thresholded_image_coloured, cmap='gray')
plt.title('Thresholded Coloured Image'), plt.xticks([]), plt.yticks([])


# Show the plots
plt.show()

#### 5) Model Training

In [ ]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)


train_images = train_images.reshape(train_images.shape[0], -1)
train_labels = train_labels.reshape(train_images.shape[0], -1)

In [ ]:
# Utilizing Gaussian Naive Bayes for classification
from sklearn.naive_bayes import GaussianNB
clf2 = GaussianNB()
train_labels = train_labels.ravel()
clf2.fit(train_images, train_labels)
predictions2 = clf2.predict(train_images)

#### 6) Evaluation Metrics

In [ ]:
# accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(train_labels, predictions2)
print('Accuracy = ', accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix

# confusion matrix
confusion_matrix = confusion_matrix(train_labels, predictions2)
confusion_matrix

In [ ]:
import seaborn as sns
sns.heatmap(confusion_matrix/np.sum(confusion_matrix), annot=True,  fmt='.2%', cmap='Blues')

In [ ]:
from sklearn.metrics import precision_score, recall_score
tp = confusion_matrix[0,0]
print("TP = ", tp)
fp = confusion_matrix[1,0]
print("FP = ", fp)
tn = confusion_matrix[1,1]
print("TN = ", tn)
fn = confusion_matrix[0,1]
print("FN = ", fn)
sensitivity = tp / (tp + fn)
print('Sensitivity = ', sensitivity)
specificity = tn/ (fp + tn)
print('Specificity = ', specificity)
precision = precision_score(train_labels, predictions2)
print('Precision = ', precision)
recall = recall_score(train_labels, predictions2)
print('Recall = ', recall)

In [ ]:
# Precision-Recall Curve

from sklearn.metrics import precision_recall_curve, average_precision_score


precision, recall, _ = precision_recall_curve(train_labels, predictions2)
average_precision = average_precision_score(train_labels, predictions2)
plt.figure()
plt.step(recall, precision, color='b', alpha=0.2, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve: AP={0:0.2f}'.format(average_precision))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Calculate predicted probabilities for the positive class
probabilities = clf2.predict_proba(train_images)[:, 1]

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(train_labels, probabilities)

# Calculate the AUC (Area Under the Curve)
roc_auc = auc(fpr, tpr)
print("Area Under Curve = ", roc_auc)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

In [ ]:
from sklearn.model_selection import cross_val_score

scores2 = cross_val_score(clf2, train_images, train_labels, scoring="accuracy", cv=10)

def display_scores(scores):
    print("Scores:", scores2)
    print("Mean:", scores2.mean())
    print("Standard deviation:", scores2.std())

display_scores(scores2)

In [ ]:
scores2 = cross_val_score(clf2, train_images, train_labels, scoring="neg_mean_squared_error", cv=10)
clf2_rmse_scores = np.sqrt(-scores2)

display_scores(clf2_rmse_scores)

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(train_labels, predictions2)

print(report)

In [ ]:
tp = confusion_matrix.diagonal()
fn = np.sum(confusion_matrix, axis=1) - tp
fp = np.sum(confusion_matrix, axis=0) - tp
tn = np.sum(confusion_matrix) - (tp + fn + fp)

# TP Rate (Sensitivity or Recall)
tp_rate = tp / (tp + fn)

# FP Rate
fp_rate = fp / (fp + tn)

for class_label, tp, fp in zip(range(len(tp_rate)), tp_rate, fp_rate):
    print(f"Class {class_label}: \nTP Rate = {tp}, \nFP Rate = {fp}")

In [ ]:
specificity = []
unique_labels = np.unique(train_labels)

for i in range(len(unique_labels)):
    true_negative = np.sum(confusion_matrix) - np.sum(confusion_matrix[i, :]) - np.sum(confusion_matrix[:, i]) + confusion_matrix[i, i]
    total_negative = np.sum(confusion_matrix) - np.sum(confusion_matrix[i, :])
    print(f'Label {unique_labels[i]} specificity: {true_negative / total_negative}')

In [ ]:
from sklearn.metrics import recall_score

sensitivity = recall_score(train_labels, predictions2, average=None)
for i in range(len(unique_labels)):
    print(f'Label {unique_labels[i]} sensitivity: {sensitivity[i]}')

In [ ]:
#plotting ROC curve for each class

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

probabilities = clf2.predict_proba(train_images)
roc_auc_scores = []
plt.figure(figsize=(8, 6))

for i in range(len(unique_labels)):
    fpr, tpr, _ = roc_curve((train_labels == unique_labels[i]).astype(int), probabilities[:, i])
    roc_auc = auc(fpr, tpr)
    roc_auc_scores.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
    
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

##### Implemented PCA:

In [ ]:
from sklearn.decomposition import PCA
  
# Reduce to 5 features
num_components = 5 
pca = PCA(num_components)
train_reduced5 = pca.fit_transform(train_images)
# Reduce to 10 features
num_components = 10 
pca = PCA(num_components)
train_reduced10 = pca.fit_transform(train_images)
# Reduce to 20 features
num_components = 20
pca = PCA(num_components)
train_reduced20 = pca.fit_transform(train_images)

train_labels = pd.DataFrame(train_labels)

result5 = pd.concat([train_labels, pd.DataFrame(train_reduced5,columns=['PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5'])], axis=1)
print(result5)
result10 = pd.concat([train_labels, pd.DataFrame(train_reduced10,columns=['PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10'])], axis=1)
print(result10)
result20 = pd.concat([train_labels, pd.DataFrame(train_reduced20,columns=['PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10', 'PCA11', 'PCA12', 'PCA13', 'PCA14', 'PCA15', 'PCA16', 'PCA17', 'PCA18', 'PCA19', 'PCA20'])], axis=1)
print(result20)

In [ ]:
from sklearn.naive_bayes import GaussianNB

def evaluate_gaussian_nb(X_train, y_train):
    nb_model = GaussianNB()
    nb_model.fit(X_train, y_train)
    y_pred = nb_model.predict(X_train)
    accuracy = accuracy_score(y_train, y_pred)
    precision = precision_score(y_train, y_pred)
    recall = recall_score(y_train, y_pred)
    f1 = f1_score(y_train, y_pred)
    roc_auc = roc_auc_score(y_train, y_pred)
    cross_val_scores = cross_val_score(nb_model, X_train, y_train, scoring="accuracy", cv=10)
    return accuracy, precision, recall, f1, roc_auc, cross_val_scores

In [ ]:
accuracies = []

accuracy, precision, recall, f1, roc_auc, cross_val_scores = evaluate_gaussian_nb(result5[['PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5']], result5[0])
accuracies.append(accuracy)
print("Accuracy = ", accuracy)
print("Precision = ", precision)
print("Recall = ", recall)
print("F1 = ", f1)
print("ROC AUC = ", roc_auc)
print("Cross Validation Score = ", cross_val_scores)

In [ ]:
accuracy, precision, recall, f1, roc_auc, cross_val_scores = evaluate_gaussian_nb(result10[['PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10']], result10[0])
accuracies.append(accuracy)
print("Accuracy = ", accuracy)
print("Precision = ", precision)
print("Recall = ", recall)
print("F1 = ", f1)
print("ROC AUC = ", roc_auc)
print("Cross Validation Score = ", cross_val_scores)

In [ ]:
accuracy, precision, recall, f1, roc_auc, cross_val_scores = evaluate_gaussian_nb(result20[['PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10', 'PCA11', 'PCA12', 'PCA13', 'PCA14', 'PCA15', 'PCA16', 'PCA17', 'PCA18', 'PCA19', 'PCA20']], result20[0])
accuracies.append(accuracy)
print("Accuracy = ", accuracy)
print("Precision = ", precision)
print("Recall = ", recall)
print("F1 = ", f1)
print("ROC AUC = ", roc_auc)
print("Cross Validation Score = ", cross_val_scores)

In [ ]:
print('Mean Accuracy = ', np.mean(accuracies))